In [1]:
import pyspark.sql
import pyspark.sql.functions as sf

from pyspark.sql.window import Window

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("best_one").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [2]:
data = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .option("sep", ',') \
    .csv("WA_Sales_Products_2012-14_Updated.csv")

In [4]:
data.show(1, True, True)

-RECORD 0---------------------------------
 Country           | United States        
 Order method type | Fax                  
 Retailer type     | Outdoors Shop        
 Product line      | Camping Equipment    
 Item Type         | Cooking Gear         
 Product           | TrailChef Deluxe ... 
 Year              | 2012                 
 Quarter           | Q1 2012              
 Total Revenue     | 59628.66             
 Units Sold        | 489                  
 Unit Cost         | 121.94               
 Gross margin      | 0.34754797           
 Total Profit      | -18233.4345          
only showing top 1 row



### Вычисления без оконных функций

In [7]:
aggregated_data = data.groupBy(
    "Country",
    "Retailer type",
    "Product line",
    "Quarter"
).agg(
    sf.sum("Total Revenue").alias("Revenue"),
    sf.sum("Units Sold").alias("Quantity")
)

aggregated_data.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: long (nullable = true)



In [ ]:
# будем искать Revenue для разных кварталов

In [8]:
extended_data = aggregated_data.select(
    sf.col("*"),
    sf.substring(aggregated_data["Quarter"],1,2).alias("q"),
    sf.substring(aggregated_data["Quarter"],3,8).alias("y")
)

extended_data.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- q: string (nullable = true)
 |-- y: string (nullable = true)



In [12]:
# расчет среднего Revenue
avg_data = extended_data.groupBy(
        "Country",
        "Retailer type",
        "Product line"
    ).agg(
        sf.avg("Revenue").alias("avg_revenue")
    )

avg_data.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Retailer type: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- avg_revenue: double (nullable = true)



In [15]:
# соединение всех DF и вычисление результата
result = extended_data.join(
        avg_data,
        (extended_data["Country"] == avg_data["Country"]) &
        (extended_data["Retailer type"] == avg_data["Retailer type"]) &
        (extended_data["Product line"] == avg_data["Product line"])
    ).select(
        extended_data["*"],
        (extended_data["Revenue"] - avg_data["avg_revenue"]).alias("revenue_delta")
    )

sorted_result = result \
    .orderBy("Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y")

sorted_result.limit(10).toPandas()

,Country,Retailer type,Product line,Quarter,Revenue,Quantity,revenue_delta
0,Australia,Department Store,Camping Equipment,Q1 2012,780636.36,23378,-6.546596e+05
1,Australia,Department Store,Camping Equipment,Q2 2012,1022203.02,15407,-4.130929e+05
2,Australia,Department Store,Camping Equipment,Q3 2012,904059.77,18591,-5.312361e+05
3,Australia,Department Store,Camping Equipment,Q4 2012,1074038.59,24115,-3.612573e+05
4,Australia,Department Store,Camping Equipment,Q1 2013,1324342.98,35357,-1.109529e+05
5,Australia,Department Store,Camping Equipment,Q2 2013,2545450.20,49263,1.110154e+06
6,Australia,Department Store,Camping Equipment,Q3 2013,2116205.77,34150,6.809099e+05
7,Australia,Department Store,Camping Equipment,Q4 2013,1515953.18,38685,8.065727e+04
8,Australia,Department Store,Camping Equipment,Q1 2014,1568783.04,35739,1.334871e+05
9,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056,1.171662e+06


In [16]:
# обзор плана вычислений
result.explain(True)

== Parsed Logical Plan ==
'Project [ResolvedStar(Country#16, Retailer type#18, Product line#19, Quarter#23, Revenue#173, Quantity#175L, q#182, y#183), (Revenue#173 - avg_revenue#231) AS revenue_delta#416]
+- Join Inner, (((Country#16 = Country#331) AND (Retailer type#18 = Retailer type#333)) AND (Product line#19 = Product line#334))
   :- Project [Country#16, Retailer type#18, Product line#19, Quarter#23, Revenue#173, Quantity#175L, substring(Quarter#23, 1, 2) AS q#182, substring(Quarter#23, 3, 8) AS y#183]
   :  +- Aggregate [Country#16, Retailer type#18, Product line#19, Quarter#23], [Country#16, Retailer type#18, Product line#19, Quarter#23, sum(Total Revenue#24) AS Revenue#173, sum(cast(Units Sold#25 as bigint)) AS Quantity#175L]
   :     +- Relation[Country#16,Order method type#17,Retailer type#18,Product line#19,Item Type#20,Product#21,Year#22,Quarter#23,Total Revenue#24,Units Sold#25,Unit Cost#26,Gross margin#27,Total Profit#28] csv
   +- Aggregate [Country#331, Retailer type#33

# Используем Windowing

аналогично с функцией из SQL

вычисляет не только sum, min, но и

- rank()
- dense_rank()
- row_number()
- lag(column, n) and lead(column, n)

In [18]:
extended_data = aggregated_data.select(
    sf.col("*"),
    sf.substring(aggregated_data["Quarter"],1,2).alias("q"),
    sf.substring(aggregated_data["Quarter"],3,8).alias("y")
)

In [24]:
# создадим окно
avg_window = Window\
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing) \
    .partitionBy(
        "Country",
        "Retailer type",
        "Product line"
    )

In [25]:
# аналитика с оконной функцией
result = extended_data.select(
        sf.col("*"),
        sf.avg(extended_data["Revenue"]).over(avg_window).alias("avg_revenue"),
    ) \
    .select(
        sf.col("*"),
        (sf.col("Revenue") - sf.col("avg_revenue")).alias("revenue_diff")
    )

# результат
sorted_result = result \
    .orderBy("Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y")

sorted_result.limit(10).toPandas()

,Country,Retailer type,Product line,Quarter,Revenue,Quantity,avg_revenue,revenue_diff
0,Australia,Department Store,Camping Equipment,Q1 2012,780636.36,23378,1.435296e+06,-6.546596e+05
1,Australia,Department Store,Camping Equipment,Q2 2012,1022203.02,15407,1.435296e+06,-4.130929e+05
2,Australia,Department Store,Camping Equipment,Q3 2012,904059.77,18591,1.435296e+06,-5.312361e+05
3,Australia,Department Store,Camping Equipment,Q4 2012,1074038.59,24115,1.435296e+06,-3.612573e+05
4,Australia,Department Store,Camping Equipment,Q1 2013,1324342.98,35357,1.435296e+06,-1.109529e+05
5,Australia,Department Store,Camping Equipment,Q2 2013,2545450.20,49263,1.435296e+06,1.110154e+06
6,Australia,Department Store,Camping Equipment,Q3 2013,2116205.77,34150,1.435296e+06,6.809099e+05
7,Australia,Department Store,Camping Equipment,Q4 2013,1515953.18,38685,1.435296e+06,8.065727e+04
8,Australia,Department Store,Camping Equipment,Q1 2014,1568783.04,35739,1.435296e+06,1.334871e+05
9,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056,1.435296e+06,1.171662e+06


In [27]:
result.explain(True)

== Parsed Logical Plan ==
'Project [*, ('Revenue - 'avg_revenue) AS revenue_diff#481]
+- Project [Country#16, Retailer type#18, Product line#19, Quarter#23, Revenue#173, Quantity#175L, q#454, y#455, avg_revenue#471]
   +- Project [Country#16, Retailer type#18, Product line#19, Quarter#23, Revenue#173, Quantity#175L, q#454, y#455, avg_revenue#471, avg_revenue#471]
      +- Window [avg(Revenue#173) windowspecdefinition(Country#16, Retailer type#18, Product line#19, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS avg_revenue#471], [Country#16, Retailer type#18, Product line#19]
         +- Project [Country#16, Retailer type#18, Product line#19, Quarter#23, Revenue#173, Quantity#175L, q#454, y#455]
            +- Project [Country#16, Retailer type#18, Product line#19, Quarter#23, Revenue#173, Quantity#175L, substring(Quarter#23, 1, 2) AS q#454, substring(Quarter#23, 3, 8) AS y#455]
               +- Aggregate [Country#16, Retailer type#18, Product line#19, 

In [ ]:
# Another interesting question would be, which quarter was the best one in each country for each retailer type and product line. This would be already much harder to do with a join, since the join key would probably need to contain the maximum revenue, which is a double (never join on floating point values, it might not work).

In [28]:
# Define a ranking window
rank_window = Window\
    .orderBy(extended_data["Revenue"].desc())\
    .partitionBy(
        "Country",
        "Retailer type",
        "Product line"
    )

In [29]:
# Perform analysis using the "row_number" window function
ranked_data = extended_data.select(
        sf.col("*"),
        sf.row_number().over(rank_window).alias("rank")
    )

# Pick the top entry of every window by filtering on the row number
result = ranked_data.filter(sf.col("rank") == 1)

# Sort result, just to improve output
sorted_result = result \
    .orderBy("Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y", "rank")

sorted_result.limit(10).toPandas()

,Country,Retailer type,Product line,Quarter,Revenue,Quantity
0,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056
1,Australia,Department Store,Golf Equipment,Q2 2014,677267.68,4916
2,Australia,Department Store,Outdoor Protection,Q4 2012,78711.97,13393
3,Australia,Department Store,Personal Accessories,Q2 2014,819106.85,18057
4,Australia,Direct Marketing,Camping Equipment,Q1 2013,588239.09,16345
5,Australia,Direct Marketing,Golf Equipment,Q1 2014,24586.64,275
6,Australia,Direct Marketing,Outdoor Protection,Q3 2013,49510.36,7455
7,Australia,Direct Marketing,Personal Accessories,Q3 2013,160893.21,4529
8,Australia,Equipment Rental Store,Camping Equipment,Q3 2013,187676.43,3438
9,Australia,Equipment Rental Store,Golf Equipment,Q3 2013,107589.55,192


In [ ]:
# Calc diff

In [30]:
extended_data = aggregated_data.select(
    sf.col("*"),
    sf.substring(aggregated_data["Quarter"],1,2).alias("q"),
    sf.substring(aggregated_data["Quarter"],3,8).alias("y")
)

In [32]:
prev_window = Window \
    .orderBy(extended_data["y"].asc(),extended_data["q"].asc())\
    .rowsBetween(-1, -1) \
    .partitionBy(
        "Country",
        "Retailer type",
        "Product line"
    )

In [34]:
result = extended_data.select(
        sf.col("*"),
        (extended_data["Revenue"] - sf.lag(extended_data["Revenue"], 1).over(prev_window)).alias("revenue_delta")
    )

sorted_result = result \
    .orderBy("Country", "Retailer Type", "Product line", "y", "q") \
    .drop("q", "y")

sorted_result.limit(10).toPandas()

,Country,Retailer type,Product line,Quarter,Revenue,Quantity,revenue_delta
0,Australia,Department Store,Camping Equipment,Q1 2012,780636.36,23378,NaN
1,Australia,Department Store,Camping Equipment,Q2 2012,1022203.02,15407,241566.66
2,Australia,Department Store,Camping Equipment,Q3 2012,904059.77,18591,-118143.25
3,Australia,Department Store,Camping Equipment,Q4 2012,1074038.59,24115,169978.82
4,Australia,Department Store,Camping Equipment,Q1 2013,1324342.98,35357,250304.39
5,Australia,Department Store,Camping Equipment,Q2 2013,2545450.20,49263,1221107.22
6,Australia,Department Store,Camping Equipment,Q3 2013,2116205.77,34150,-429244.43
7,Australia,Department Store,Camping Equipment,Q4 2013,1515953.18,38685,-600252.59
8,Australia,Department Store,Camping Equipment,Q1 2014,1568783.04,35739,52829.86
9,Australia,Department Store,Camping Equipment,Q2 2014,2606957.75,41056,1038174.71
